In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import json
from rich.progress import Progress, TextColumn, BarColumn, SpinnerColumn, MofNCompleteColumn
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import wandb
from torch.optim.lr_scheduler import ExponentialLR

In [ ]:
MODEL_NAME = "FacebookAI/roberta-base"

In [3]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")
wandb.login()

wandb: Currently logged in as: shobhitshukla6535 (shobhitshukla6535-iit-kharagpur) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
class QuestionsDataset(Dataset):
    def __init__(self, dataset, tokenizer, is_test=False, max_length=62):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_test = is_test

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = str(self.dataset.iloc[idx, 1])

        if not self.is_test:
            target = self.dataset.iloc[idx, 2]
            # assert isinstance(target, np.int16)
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            add_special_tokens=True,
            return_tensors="pt",
        )

        if self.is_test:
            return {
                "input_ids": encoding["input_ids"].squeeze(),
                "attention_mask": encoding["attention_mask"].squeeze(),
            }

        else:
            return {
                "input_ids": encoding["input_ids"].squeeze(),
                "attention_mask": encoding["attention_mask"].squeeze(),
                "targets": torch.FloatTensor([target]),
            }

In [5]:
class BERT_MODEL(nn.Module):
    def __init__(self, model_name):
        super(BERT_MODEL, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.linear = nn.Sequential(
            nn.Linear(768, 1024), nn.ReLU(), nn.Dropout(0.3), nn.Linear(1024, 1)
        )

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = output.pooler_output
        output = self.linear(pooled_output)
        return output

In [ ]:
class Trainer:
    def __init__(
        self,
        dataset_path,
        model_card,
        checkpoint_dir="checkpoints",
        device="cuda" if torch.cuda.is_available() else "cpu",
        pos_weight=[1, 70],
        length_percentile=99.9,
        batch_size=32,
        epochs=2,
        seed=42,
    ):
        self.device = device
        self.batch_size = batch_size
        self.epochs = epochs
        self.seed = seed
        print(f"Using {self.device} device")
        self.model_name = model_card
        self.get_dataloader(dataset_path, length_percentile)
        print(f"Train dataset size: {len(self.train_dl.dataset)}")
        print(f"Validation dataset size: {len(self.val_dl.dataset)}")
        self.model = BERT_MODEL(self.model_name)
        print(f"Model: {self.model_name} initialized")
        self.num_epochs = epochs
        self.criterion = nn.BCEWithLogitsLoss()
        self.checkpoint_dir = checkpoint_dir
        os.makedirs(checkpoint_dir, exist_ok=True)
        self.optimizer = optim.AdamW(self.model.parameters(), lr=1e-5)
        self.scheduler = ExponentialLR(self.optimizer, gamma=0.9)

        wandb.init(
            project="question-classification",
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "learning_rate": 1e-5,
                "model": self.model_name,
            },
        )

    @staticmethod
    def get_max_length(df_train, tokenizer, length_percentile=99.9):
        df_train["question_length"] = tokenizer(
            df_train.question_text.tolist(), truncation=True
        )["input_ids"]
        df_train["question_length"] = df_train["question_length"].apply(
            lambda x: len(x)
        )
        max_length = np.percentile(df_train["question_length"], length_percentile)

        return int(max_length)

    def get_dataloader(self, path, length_percentile=99.9):
        df_train = pd.read_csv(path)
        df_train.target = df_train.target.astype("int16")
        # df_train = df_train.iloc[:100,:]
        tokenizer = AutoTokenizer.from_pretrained(self.model_name, use_fast=True)
        max_length = 62  # self.get_max_length(df_train, tokenizer, length_percentile)
        train_df, val_df = train_test_split(
            df_train,
            stratify=df_train.target,
            test_size=0.2,
            random_state=42,
        )

        train_ds = QuestionsDataset(
            train_df, tokenizer, is_test=False, max_length=max_length
        )
        val_ds = QuestionsDataset(
            val_df, tokenizer, is_test=False, max_length=max_length
        )

        self.train_dl = DataLoader(train_ds, batch_size=self.batch_size, shuffle=True)
        self.val_dl = DataLoader(val_ds, batch_size=self.batch_size)

    @staticmethod
    def find_best_f1(outputs, labels):
        tmp = [0, 0, 0]  # idx, current, max
        threshold = 0

        for tmp[0] in np.arange(0.25, 0.85, 0.01):
            tmp[1] = f1_score(labels, outputs > tmp[0], zero_division=0)
            if tmp[1] > tmp[2]:
                threshold = tmp[0]
                tmp[2] = tmp[1]

        return tmp[2], threshold

    def train(self):
        torch.manual_seed(self.seed)
        self.model.to(self.device)
        history = {"Train_Loss": [], "Val_Loss": [], "F1": []}
        agg_loss = 0
        step = 0

        progress = Progress(
            TextColumn("[bold blue]{task.description}"),
            SpinnerColumn(),
            BarColumn(),
            MofNCompleteColumn(),
            TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
        )

        epoch_progress = progress.add_task(
            "[cyan] Epochs...: ", total=self.num_epochs, start=True
        )

        for epoch in range(self.num_epochs):
            progress.update(epoch_progress, advance=1)
            batch_progress = progress.add_task(
                "[blue] Training :", total=len(self.train_dl)
            )

            for i, batch in enumerate(self.train_dl):
                input_ids, attention_mask, targets = (
                    batch["input_ids"].to(self.device),
                    batch["attention_mask"].to(self.device),
                    batch["targets"].to(self.device).squeeze(1),
                )

                self.optimizer.zero_grad()
                logits = self.model(input_ids, attention_mask)
                loss = self.criterion(logits.squeeze(1), targets)
                agg_loss += loss.item()
                loss.backward()

                nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

                self.optimizer.step()

                history["Train_Loss"].append(agg_loss / (step + 1))
                wandb.log({"Train Loss": history["Train_Loss"][-1]}, step=step + 1)

                progress.update(
                    batch_progress,
                    advance=1,
                    description=f"Train loss: {history['Train_Loss'][-1]:.4f}",
                )

                step = step + 1

                if (step + 1) % 2000 == 0:
                    checkpoint_path = os.path.join(
                        self.checkpoint_dir, f"model_epoch_{epoch + 1}.pth"
                    )
                    self.scheduler.step()

                    wandb.log({"last_lr": self.scheduler.get_last_lr()[0]}, step=step)

                    torch.save(
                        {
                            "epoch": epoch + 1,
                            "model_state_dict": self.model.state_dict(),
                            "optimizer_state_dict": self.optimizer.state_dict(),
                            "loss": history["Train_Loss"][-1],
                        },
                        checkpoint_path,
                    )
                    progress.console.print(f"Checkpoint saved at {checkpoint_path}")
                    val_loss, best_f1, threshold = self.validate(progress)
                    progress.console.print(
                        f"Learning Rate after {step} steps: {self.scheduler.get_last_lr()[0]} \n",
                        f"Train loss: {history['Train_Loss'][-1]:.4f} \n",
                        f"Validation Loss: {val_loss}, Best F1 Score: {best_f1}, Threshold: {threshold}\n",
                    )
                    wandb.log(
                        {
                            "Validation Loss": val_loss,
                            "F1 Score": best_f1,
                            "at_threshold": threshold,
                        },
                        step=step,
                    )
                    history["Val_Loss"].append(val_loss)
                    history["F1"].append(best_f1)

            progress.remove_task(batch_progress)

        history_path = os.path.join(self.checkpoint_dir, "training_history.json")
        with open(history_path, "w") as f:
            json.dump(history, f)

        wandb.save(history_path)

        model_path = os.path.join(self.checkpoint_dir, "final_model.pth")
        torch.save(self.model.state_dict(), model_path)

        wandb.save(model_path)

        print("Training complete! History and model saved to W&B.")
        wandb.finish()
        return history

    def validate(self, progress):
        """Validation logic separated into its own function."""
        self.model.eval()
        agg_val_loss = 0
        all_logits, all_targets = [], []
        val_progress = progress.add_task("[Red] Validating :", total=len(self.val_dl))
        with torch.no_grad():
            for i, batch in enumerate(self.val_dl):
                input_ids, attention_mask, targets = (
                    batch["input_ids"].to(self.device),
                    batch["attention_mask"].to(self.device),
                    batch["targets"].to(self.device).squeeze(1),
                )

                logits = self.model(input_ids, attention_mask)
                loss = self.criterion(logits.squeeze(1), targets)
                agg_val_loss += loss.item()
                progress.update(
                    val_progress,
                    advance=1,
                    description=f"Validated {i + 1} Batches",
                )
                all_logits.append(logits.squeeze(1).cpu())
                all_targets.append(targets.cpu())

        all_logits = torch.cat(all_logits, dim=0)
        all_targets = torch.cat(all_targets, dim=0)

        val_loss = agg_val_loss / len(self.val_dl)
        best_f1, threshold = self.find_best_f1(
            torch.sigmoid(all_logits).numpy(), all_targets.numpy()
        )

        return val_loss, best_f1, threshold

In [ ]:
trainer = Trainer(
    dataset_path="/kaggle/input/quora-insincere-questions-classification/train.csv",
    model_card=MODEL_NAME,
    checkpoint_dir="checkpoints",
    pos_weight=[1, 20],
    length_percentile=99.9,
    batch_size=64,
    epochs=2
)

In [ ]:
history = trainer.train()

In [ ]:
from time import sleep
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn


# Validation function with progress bar
def validate(progress, total_batches=50):
    validation_task = progress.add_task("[magenta]Validating...", total=total_batches)

    for _ in range(total_batches):
        sleep(0.01)  # Simulate validation step
        progress.update(validation_task, advance=1)

    progress.remove_task(validation_task)


# Training function with integrated progress bars
def train():
    epochs = 3
    total_batches = 500
    validate_every = 50
    validation_batches = 50

    with Progress(
        TextColumn("[bold blue]{task.description}"),
        BarColumn(),
        TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
    ) as progress:
        epoch_task = progress.add_task("[green]Epoch Progress", total=epochs)

        for epoch in range(epochs):
            batch_task = progress.add_task(
                "[cyan]Training Batches", total=total_batches
            )

            print(f"Working on epoch #{epoch}")

            for batch in range(1, total_batches + 1):
                sleep(0.01)  # Simulate training step
                progress.update(batch_task, advance=1)

                if batch % validate_every == 0:
                    validate(progress, validation_batches)  # Call validate function

            progress.update(epoch_task, advance=1)
            progress.remove_task(batch_task)


# Run training
train()

Output()

Working on epoch #0

Working on epoch #1

Working on epoch #2

KeyboardInterrupt: 